### Import libraries

In [1]:
import pickle
import pandas as pd
from numpy import arange
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report, make_scorer, recall_score
from constants import target_col as target_name, index_col as index_name, model_out_path, report_cmap, seed, test_size
from utilities import do_modeling, get_evaluation_report

### Configure defaults

In [2]:
val_size = test_size
prediction_column = 'prediction'

index_col = index_name.lower()
target_col = [target_name]

plt.style.use('fivethirtyeight')

### Read data

In [3]:
train_std = pd.read_csv('./outputs/std/train.csv').set_index(index_col)
test_std = pd.read_csv('./outputs/std/test.csv').set_index(index_col)

train_mm = pd.read_csv('./outputs/min_max/train.csv').set_index(index_col)
test_mm = pd.read_csv('./outputs/min_max/test.csv').set_index(index_col)

### Data check

In [4]:
train_std.head()

,attrition_flag,gender_F,gender_M,customer_age,dependent_count,months_on_book,total_relationship_count,months_inactive_12_mon,contacts_count_12_mon,credit_limit,total_revolving_bal,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio
clientnum,,,,,,,,,,,,,,,,
713796033,1,0,1,0.085971,-0.271041,0.761682,1.409797,0.661411,0.498335,-0.427523,-1.423834,-0.541924,-0.736571,-1.228105,-0.892927,-0.997729
771204108,0,1,0,-1.290336,-0.271041,-1.623249,-0.527633,-1.331855,-2.214038,-0.734047,0.023954,0.489055,-0.149068,0.092368,0.025604,1.201739
721399833,0,1,0,-0.414504,-0.271041,-0.242499,-0.527633,-1.331855,0.498335,-0.725459,-0.186031,0.543798,0.098177,0.901690,0.497510,0.811452
778601733,1,1,0,1.086921,0.499385,1.389296,0.763987,1.658044,0.498335,-0.653342,0.650224,-0.208908,-0.598231,-1.313297,0.324759,1.296575
714047733,0,1,0,0.836684,0.499385,1.138250,-1.173443,-0.335222,-1.309914,-0.405063,1.178255,-0.190661,0.899960,0.859094,-0.454728,0.567066


In [5]:
train_mm.head()

,attrition_flag,gender_F,gender_M,customer_age,dependent_count,months_on_book,total_relationship_count,months_inactive_12_mon,contacts_count_12_mon,credit_limit,total_revolving_bal,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio
clientnum,,,,,,,,,,,,,,,,
713796033,1,0,1,0.446809,0.4,0.674419,1.0,0.500000,0.500000,0.099756,0.000000,0.188696,0.076306,0.201550,0.140017,0.000000
771204108,0,1,0,0.212766,0.4,0.232558,0.4,0.166667,0.000000,0.015591,0.468415,0.255225,0.187479,0.441860,0.201064,0.603604
721399833,0,1,0,0.361702,0.4,0.488372,0.4,0.166667,0.500000,0.017949,0.400477,0.258758,0.234265,0.589147,0.232428,0.496496
778601733,1,1,0,0.617021,0.6,0.790698,0.8,0.666667,0.500000,0.037751,0.671037,0.210185,0.102484,0.186047,0.220947,0.629630
714047733,0,1,0,0.574468,0.6,0.744186,0.2,0.333333,0.166667,0.105923,0.841875,0.211363,0.385986,0.581395,0.169140,0.429429


In [6]:
print('Standardized train set shape:', train_std.shape, 'test set shape:', test_std.shape)
print('MinMax scaled train set shape:', train_mm.shape, 'test set shape:', test_mm.shape)

Standardized train set shape: (8607, 16) test set shape: (1520, 16)
MinMax scaled train set shape: (8607, 16) test set shape: (1520, 16)


In [7]:
print('NA count by column and dataset')

NA_df = pd.DataFrame()
NA_df['train_standardized'] = train_std.isna().sum()
NA_df['test_standardized'] = test_std.isna().sum()
NA_df['train_minmax'] = train_mm.isna().sum()
NA_df['test_minmax'] = test_mm.isna().sum()
NA_df.T

NA count by column and dataset


,attrition_flag,gender_F,gender_M,customer_age,dependent_count,months_on_book,total_relationship_count,months_inactive_12_mon,contacts_count_12_mon,credit_limit,total_revolving_bal,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio
train_standardized,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
test_standardized,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
train_minmax,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
test_minmax,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Observation**
- Dataset has `16` columns (target included) as well it is divided into train set with `8607` `(85 %)` rows and test set with `1520` `(15 %)` rows
- There are no missing values occurred from bug during preprocessing in both training sets and testing sets

### Separate target

In [8]:
std_train, std_val = train_test_split(train_std, test_size=val_size, random_state=seed, stratify=train_std[target_name])
mm_train, mm_val = train_test_split(train_mm, test_size=val_size, random_state=seed, stratify=train_mm[target_name])

X_train_std, y_train_std = std_train.drop(columns=target_col), std_train[target_name]
X_val_std, y_val_std = std_val.drop(columns=target_col), std_val[target_name]
X_test_std, y_test_std = test_std.drop(columns=target_col), test_std[target_name]

X_train_mm, y_train_mm = mm_train.drop(columns=target_col), mm_train[target_name]
X_val_mm, y_val_mm = mm_val.drop(columns=target_col), mm_val[target_name]
X_test_mm, y_test_mm = test_mm.drop(columns=target_col), test_mm[target_name]

### Modeling

In [9]:
std_models = do_modeling(X_train_std, y_train_std)
mm_models = do_modeling(X_train_mm, y_train_mm)

In [10]:
std_models, mm_models

({'Decision Tree': DecisionTreeClassifier(class_weight='balanced', random_state=42),
  'Random Forest': RandomForestClassifier(class_weight='balanced', random_state=42),
  'Logistic Regression': LogisticRegression(class_weight='balanced', random_state=42),
  'Support Vector Machine': SVC(class_weight='balanced', random_state=42)},
 {'Decision Tree': DecisionTreeClassifier(class_weight='balanced', random_state=42),
  'Random Forest': RandomForestClassifier(class_weight='balanced', random_state=42),
  'Logistic Regression': LogisticRegression(class_weight='balanced', random_state=42),
  'Support Vector Machine': SVC(class_weight='balanced', random_state=42)})

### Evaluation

In [11]:
std_eval_report = get_evaluation_report(std_models, X_val_std, y_val_std)
mm_eval_report = get_evaluation_report(mm_models, X_val_mm, y_val_mm)

In [12]:
std_eval_report.style.background_gradient(cmap=report_cmap)

,algorithm,class,precision,recall,f1,support
0,Decision Tree,0,0.961644,0.971402,0.966498,1084
1,Decision Tree,1,0.842640,0.798077,0.819753,208
2,Random Forest,0,0.956250,0.988007,0.971869,1084
3,Random Forest,1,0.924419,0.764423,0.836842,208
4,Logistic Regression,0,0.966245,0.845018,0.901575,1084
5,Logistic Regression,1,0.511628,0.846154,0.637681,208
6,Support Vector Machine,0,0.980178,0.912362,0.945055,1084
7,Support Vector Machine,1,0.664311,0.903846,0.765784,208


In [13]:
mm_eval_report.style.background_gradient(cmap=report_cmap)

,algorithm,class,precision,recall,f1,support
0,Decision Tree,0,0.961644,0.971402,0.966498,1084
1,Decision Tree,1,0.842640,0.798077,0.819753,208
2,Random Forest,0,0.955437,0.988930,0.971895,1084
3,Random Forest,1,0.929412,0.759615,0.835979,208
4,Logistic Regression,0,0.963041,0.841328,0.898080,1084
5,Logistic Regression,1,0.501449,0.831731,0.625678,208
6,Support Vector Machine,0,0.974773,0.891144,0.931084,1084
7,Support Vector Machine,1,0.607973,0.879808,0.719057,208


**Observation**
- For churn prediction, recall is considered as first choice of metric for model evaluation to select model with maximum ability to detect churn customers
- Support vector machine (SVM) algorithm with input standardization preprocessing has outperformed other models in churn prediction task on this dataset. Hence, SVM algorithm will be furthur tuned

### Tuning

In [14]:
params = dict(
    C=10 ** arange(-4, 2.5, 0.5)
)

scorer = make_scorer(recall_score, average='binary')

clf = RandomizedSearchCV(
    SVC(class_weight='balanced', random_state=seed),
    params,
    random_state=seed,
    cv=5,
    n_jobs=-1,
    scoring=scorer,
    n_iter=13
)

In [15]:
X_train_val_std = pd.concat([X_train_std, X_val_std])
y_train_val_std = pd.concat([y_train_std, y_val_std])

fitted = clf.fit(X_train_val_std, y_train_val_std)

In [16]:
fitted.best_estimator_

SVC(C=0.31622776601683794, class_weight='balanced', random_state=42)

In [17]:
fitted.best_score_

0.9088892376916234

### Evaluation

In [18]:
preds = fitted.best_estimator_.predict(X_test_std)
print(classification_report(y_test_std, preds))

              precision    recall  f1-score   support

           0       0.98      0.90      0.94      1276
           1       0.63      0.90      0.74       244

    accuracy                           0.90      1520
   macro avg       0.80      0.90      0.84      1520
weighted avg       0.92      0.90      0.90      1520



In [19]:
train_preds = fitted.best_estimator_.predict(X_train_val_std)
print(classification_report(y_train_val_std, train_preds))

              precision    recall  f1-score   support

           0       0.99      0.91      0.95      7224
           1       0.67      0.94      0.78      1383

    accuracy                           0.92      8607
   macro avg       0.83      0.93      0.87      8607
weighted avg       0.94      0.92      0.92      8607



**Observation**
- Parameter tuning helped obtaining a model that is better than model with default settings
- Model performance on testing set (first report) is not too far from training set (last report). Hence, it can be said that model is not memorizing training data

### Prediction

In [20]:
X_test_std[prediction_column] = preds

In [21]:
pd.concat([X_test_std[prediction_column], y_test_std], axis=1).sample(15)

,prediction,attrition_flag
clientnum,,
710451708,0,0
714914583,0,0
711838008,1,1
761240358,0,0
789481233,1,1
708794133,0,0
710822808,0,0
823606983,0,0
826478958,1,0


### Serialization

In [22]:
with open(model_out_path, "wb") as fp:
    pickle.dump(fitted.best_estimator_, fp)